In [1]:
# Copyright 2021 Fagner Cunha
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
import json
import random
import numpy as np
import pandas as pd

In [3]:
random_seed = 42

random.seed(random_seed)
np.random.seed(random_seed)

#### Loading Snaposhot Serengeti data

In [4]:
def load_instances(file_patern, splits=[], extra=[]):
    data = []
    
    for i in range(len(splits)):
        file_name = file_patern % (splits[i], extra[i])
        df = pd.read_csv(file_name)
        df['split'] = splits[i]
        data.append(df)
    
    return pd.concat(data).reset_index(drop=True)

In [5]:
serengeti_site = load_instances('/data/fagner/training/data/ss_%s_empty%s.csv',
                                ['train', 'val_dev', 'val'], ['_bal', '', ''])

In [6]:
serengeti_time = load_instances('/data/fagner/training/data/ss_time_%s_empty%s.csv',
                                ['train', 'val_dev', 'val'], ['_bal', '', ''])

In [7]:
with open('/data2/fagner/serengeti/SnapshotSerengetiBboxes_20190903.json') as json_file:
    serengeti_bboxes = json.load(json_file)

#### Mark images with bboxes

In [8]:
def add_location(row):
    return row['file_name'].split('/')[1]

In [9]:
def mark_bbox(row, bboxes_dict):
    if row['file_name'][:-4] in bboxes_dict:
        return 1
    else:
        return 0

In [10]:
bboxes_dict = {}

for bbox in serengeti_bboxes['annotations']:
    bboxes_dict[bbox['image_id']] = 1

In [11]:
serengeti_site['has_bbox'] = serengeti_site.apply(lambda row: mark_bbox(row, bboxes_dict), axis=1)
serengeti_site['location'] = serengeti_site.apply(add_location, axis=1)

In [12]:
serengeti_time['has_bbox'] = serengeti_time.apply(lambda row: mark_bbox(row, bboxes_dict), axis=1)
serengeti_time['location'] = serengeti_time.apply(add_location, axis=1)

#### Filter empty images

We limit the maximum number of empty images per location by the number of nonempty images with bounding boxes on that location. This is done on training and val_dev splits separated, validation (test) split as kept as it is.

In [13]:
def cap_empty_images_per_location(df, split):
    df = df[df.split == split]
    filtered_images = []
    
    for location in df.location.unique():
        images_w_bbox = df[(df.location == location)
                           & (df.category == 1)
                           & (df.has_bbox == 1)].copy()
        empty_images = df[(df.location == location)
                           & (df.category == 0)].copy()
        if len(empty_images) > len(images_w_bbox):
            empty_images = empty_images.sample(len(images_w_bbox)).copy()
        filtered_images.append(images_w_bbox)
        filtered_images.append(empty_images)
    
    return pd.concat(filtered_images).reset_index(drop=True)

In [14]:
def cap_empty_images(df, splits_to_cap):
    data = []
    
    for split in splits_to_cap:
        filtered_data = cap_empty_images_per_location(df, split)
        data.append(filtered_data)
    
    data.append(df[~df.split.isin(splits_to_cap)])
    
    return pd.concat(data).reset_index(drop=True)

In [15]:
filtered_serengeti_time = cap_empty_images(serengeti_time, ['train', 'val_dev'])

In [16]:
filtered_serengeti_site = cap_empty_images(serengeti_site, ['train', 'val_dev'])

#### Show quantities

In [17]:
pd.crosstab(serengeti_time.category, serengeti_time.split)

split,train,val,val_dev
category,,,
0,516635,383981,588406
1,516630,75328,225647


In [18]:
pd.crosstab(filtered_serengeti_time.category, filtered_serengeti_time.split)

split,train,val,val_dev
category,,,
0,43612,383981,18957
1,44579,75328,18957


In [19]:
pd.crosstab(serengeti_site.category, serengeti_site.split)

split,train,val,val_dev
category,,,
0,524804,535817,278578
1,523891,209183,84531


In [20]:
pd.crosstab(filtered_serengeti_site.category, filtered_serengeti_site.split)

split,train,val,val_dev
category,,,
0,51281,535817,6041
1,52081,209183,6041


#### Save list of images filtered

In [21]:
def save_to_csv(df, file_name):
    df = df[['file_name', 'category', 'split', 'location']]
    df.to_csv(file_name, index=False)

In [22]:
save_to_csv(filtered_serengeti_site, '/data/fagner/training/data/ss_all_empty_filtered.csv')
save_to_csv(filtered_serengeti_time, '/data/fagner/training/data/ss_time_all_empty_filtered.csv')